In [80]:
import pandas as pd

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/tmp/ipykernel_620854/1517969582.py:3: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('UNSPSC_English.csv')


In [81]:
#df_csv = df[8:200]
#df_csv = rename({'Unnamed: 1' : 'Key'})
#df_csv.drop('UNSPSCv23_0701')

# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df.iloc[8]).loc[9:100]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Segment', 'Family', 'Column1', 'Class', 'Synonym', 'Acronym'])
df_csv

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,NaN,NaN,NaN,NaN,NaN
11,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,NaN,NaN,NaN
12,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101501,Cats,NaN
13,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101502,Dogs,NaN
14,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101504,Mink,NaN
15,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101505,Rats,NaN
16,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101506,Horses,NaN
17,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101507,Sheep,NaN
18,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,10101508,Goats,NaN


In [82]:
#df_xlsx = df_xlsx.rename(columns=df.iloc[1], inplace=True)


df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:50]
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,Digital goods,Digital goods are generally viewed as download...
3,D9999999,Temporary unmapped digital goods sku - taxable...,A digital good that has not been mapped to a s...
4,DA010000,Digital goods: advertising material (digital o...,"Tangible personal property (i.e. designs, artw..."
5,DA030000,Digital goods: art (digital only -- no physica...,Canned artwork that is transferred to the cust...
6,DA040000,Digital goods: audio (digital only -- no physi...,Audio (i.e. music or spoken word) that is tran...
7,DA040100,Radio subscriptions,Subscriptions to satellite radio services.
8,DA050000,Digital audio works,NaN
9,DA051011,Digital audio works sold to an end user with r...,NaN
10,DA051012,Digital audio works (with rights conditioned o...,NaN
11,DA051013,Digital audio works sold to users other than t...,NaN


In [48]:
pd.set_option('display.max_columns', 17)
pd.set_option('display.max_rows', 100)

In [39]:
df = df.drop(columns='Unnamed: 16')

In [75]:
df = df.drop(index=8)

In [56]:
df = df.drop(columns='UNSPSC v23_0701')

In [76]:
df

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
9,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN
10,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,NaN,NaN,NaN,NaN
11,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,NaN,NaN
12,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Cats,NaN
13,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Dogs,NaN
...,...,...,...,...,...,...,...,...
156402,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN
156403,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Medical unit,A building or structure where medical equipmen...
156404,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Laboratory unit,A building or structure where laboratory equip...
156405,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Dental unit,A building or structure where dental equipment...


In [74]:
df = df.rename(columns={'Unnamed: 14':'Definition'})

In [79]:
df = df.reset_index()

In [81]:
df = df.drop(columns='index')

In [83]:
df.sample(n=20)

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
94577,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of lower ...,This family of surgical interventions or proce...,"Principal arteries abdomen, open approach-part F",This class includes one or more of 17 standard...,Dilation of splenic artery with drug-eluting i...,This classification denotes the group of activ...
73999,Musical Instruments and Games and Toys and Art...,This Segment includes teaching aids used in bo...,Developmental and professional teaching aids a...,NaN,Educational incentives,NaN,Bible based incentive charts,NaN
124231,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of upper ...,This family of surgical interventions or proce...,"Flat bones, percutaneous endoscopic approach-p...",This class includes one or more of 17 standard...,"Excision of left scapula, percutaneous endosc...",This classification denotes the group of activ...
139075,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of female...,This family of surgical interventions or proce...,"Uterus cervix fallopian tubes, percutaneous en...",This class includes one or more of 23 standard...,"Excision of uterus, percutaneous endoscopic a...",This classification denotes the group of activ...
155000,Healthcare Services,This segment includes services associated with...,Physical rehabilitation interventions or proce...,This family of physical rehabilitation interve...,Daily living activities-mobility and occupatio...,This class includes one or more of multiple st...,"Environmental, home and work barriers assessme...",This classification denotes the group of activ...
131537,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of upper ...,This family of surgical interventions or proce...,"Joints hand, percutaneous endoscopic approach-...",This class includes one or more of 16 standard...,Fusion of right metacarpocarpal joint with ext...,This classification denotes the group of activ...
97643,Healthcare Services,This segment includes services associated with...,Surgical interventions or procedures of lower ...,This family of surgical interventions or proce...,"Principal arteries lower extremities, percutan...",This class includes one or more of 17 standard...,Occlusion of right femoral artery with extralu...,This classification denotes the group of activ...
24444,Medical Equipment and Accessories and Supplies,"This segment includes the tools and machines, ...",Patient exam and monitoring products,"In this classification, this family includes m...",Ophthalmic diagnostic exam products,"In this classification, this class includes, p...",Ophthalmic slit lamps,"In this classification, this commodity indicat..."
68924,Drugs and Pharmaceutical Products,This segment includes natural or synthetic mat...,Antifungal drugs,This classification denotes substances that de...,Antifungal amines,Antifungal drugs made from a group of compound...,Butenafine,This classification denotes an antifungal agen...
11966,Commercial and Military and Private Vehicles a...,This segment includes vehicles used in the con...,Motor vehicles,NaN,Product and material transport vehicles,NaN,Load motovan,Special motor vehicle usually of three wheels ...


In [85]:
df.to_csv('my_DataFrame.csv')

,Segment Title,Segment Definition,Family Title,Family Definition,Class Title,Class Definition,Commodity Title,Definition
0,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN
1,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,NaN,NaN,NaN,NaN
2,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,NaN,NaN
3,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Cats,NaN
4,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",Live animals,NaN,Livestock,NaN,Dogs,NaN
...,...,...,...,...,...,...,...,...
156393,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN
156394,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Medical unit,A building or structure where medical equipmen...
156395,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Laboratory unit,A building or structure where laboratory equip...
156396,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,Prefabricated buildings and structures,Buildings and structures used for various farm...,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,Dental unit,A building or structure where dental equipment...
